In [97]:
#前処理後のindex作るための処理
import os
import elasticsearch as es
import math
import numpy as np
from itertools import chain

ES_HOST = "localhost"
escon = es.Elasticsearch(ES_HOST, port=9200)
GACRP_HOME=os.environ['GACRP_HOME']

In [98]:
# 元データ取得
body = {
    "size":10000,
    "_source":["channelGrouping","visitNumber","isMobile","country","hits","bounces","newVisits","source","medium","isTrueDirect","transactionRevenue"],
    "query": {
      "match_all": {}
    }
  }
  
res=escon.search(index='gacrp_index',body=body)
data=[]
ans = []

print(len(res["hits"]["hits"]))

10000


In [99]:
for i in range(len(res["hits"]["hits"])):
    row=res["hits"]["hits"][i]["_source"]
    rec=[]

    if row['channelGrouping']=='Display':
        col_channelGrouping=1
    elif row['channelGrouping']=='Social':
        col_channelGrouping=2
    else:
        col_channelGrouping=0
    oneHot_col_channelGrouping=np.eye(2+1)[col_channelGrouping]
        
    col_visitNumber=min(int(row['visitNumber']),5)/5.0 

    if row['isMobile']=="TRUE":
        col_isMobile=1
    else:
        col_isMobile=0

    if row['country']=="United States":
        col_country =1
    elif row['country']=="Canada":
        col_country =2
    elif row['country']=="Japan":
        col_country=3
    else:
        col_country=0
    oneHot_col_country=np.eye(3+1)[col_country]
    
    col_hits=math.log10(int(row['hits'])*1.0)
    
    if row['bounces']=="1":
        col_bounces=1
    else:
        col_bounces=0

    if row['newVisits']== "1":
        col_newVisits=1
    else:
        col_newVisits=0
    
    if row['source']=="mail.googleplex.com" or row['source']=="dfa":
        col_source =1
    elif row['source']=="(direct)":
        col_source=2
    else:
        col_source=0
    oneHot_col_source=np.eye(2+1)[col_source]
        
    if row['medium']=="cpm":
        col_medium=1
    else:
        col_medium=0
    oneHot_col_medium=np.eye(1+1)[col_medium]
    
    if row['isTrueDirect']=="1":
        col_isTrueDirect=1
    else:
        col_isTrueDirect=0
    
    revenue = math.log(int(row['transactionRevenue'])+1)
    
    l=[oneHot_col_channelGrouping,[col_visitNumber],[col_isMobile],oneHot_col_country,[col_hits],[col_bounces],[col_newVisits],oneHot_col_source,oneHot_col_medium,[col_isTrueDirect]]
    data.append(list(chain.from_iterable(l)))
    ans.append(revenue) 

In [100]:
print(len(data))
print(len(ans))

10000
10000
